In [1]:
import pydriller
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os 

In [2]:
PATH_DATASET = "../artifacts/annotation_bugs.csv"

In [7]:
from pydriller import Repository 

In [3]:
import pathlib
from pathlib import Path

OUT_ORIGINAL_COMMIT = "../artifacts/original_commits/"

In [4]:
REPO_FOLDER = "../repos"

In [5]:
df_bugs = pd.read_csv(PATH_DATASET)

In [11]:
df_bugs["only_repo_name"] = df_bugs["repo"].apply(lambda e: e.split("/")[-1])
df_bugs["msg"] = df_bugs.apply(
    lambda row: list(Repository(os.path.join(REPO_FOLDER, row["only_repo_name"]), 
                           single=row["commit_hash"]).traverse_commits())[0].msg,
    axis=1
)
df_bugs = df_bugs.sort_values(by="id")

In [12]:
df_bugs.head()

,id,real,type,repo,commit_hash,component,symptom,bug_pattern,complexity,comment,localization,only_repo_name,msg,human_id
0,1,bug,Classical,Qiskit/qiskit-ignis,ec1b4ce759f1fb8ba0242dd6c4a309fa1b586666,Quantum State Evaluation,"Crash - OS/PL Error, Crash",Missing Error Handling,19,"From fixer: ""Currently, cvx_fit does not check...",https://github.com/Qiskit/qiskit-ignis/commit/...,qiskit-ignis,Fix to allow cvx_fit to retry and finally grac...,qiskit-ignis#3
1,10,bug,Quantum,Qiskit/qiskit-ignis,e696fee3e52550f50fb240a4c6356c61c9f01725,Quantum State Evaluation,Incorrect Output,Incorrect Numerical Computation,2,"Reporter: ""ignis.characterization.calibrations...",https://github.com/Qiskit/qiskit-ignis/commit/...,qiskit-ignis,fix FC rotation direction (#302),qiskit-ignis#302
2,1010,bug,Quantum,Qiskit/qiskit-aer,dc7a2413e6c373455c00cff51e37eaaa17a4a7ac,Optimizations,"Crash - Application Error, Crash",Barrier Related,6,"Fixer: ""Currently qubit truncation will fail f...",https://github.com/Qiskit/qiskit-aer/commit/dc...,qiskit-aer,Fix ReduceBarriers to be applied before trunca...,qiskit-aer#693
3,1012,bug,Classical,Qiskit/qiskit-aer,7de864cc989b354d0ad1febbd62b891270927b6b,Testing,Failing Test,"Flaky Test, Misconfiguration, Misconfiguration...",1,"Fix issue where the ""diagonal"" gate is checked...",https://github.com/Qiskit/qiskit-aer/commit/7d...,qiskit-aer,Fix tolerance when checking if diagonal is uni...,qiskit-aer#707
4,1019,bug,Quantum,Qiskit/qiskit-aer,f3b4303f478ff84bf8da15761827287d3a2f16e4,Simulator,Incorrect Output,"API Misuse - Internal, Typo",1,"Fixer: ""The new Diagonal terra gate was being ...",https://github.com/Qiskit/qiskit-aer/commit/f3...,qiskit-aer,Fix bugs in density matrix simulation method f...,qiskit-aer#767


In [13]:
import re

used_human_ids = []

def get_human_id(short_repo_name, msg):
    global used_human_ids
    issue_id = msg.split()
    try:
        issue_id = re.search('#([0-9]+)', msg).group(0)
    except AttributeError:
        # AAA, ZZZ not found in the original string
        issue_id = None # apply your error handling
    if issue_id is not None:
        new_human_id = f"{short_repo_name}{issue_id}" 
        if not new_human_id in used_human_ids:
            used_human_ids.append(new_human_id)
            return new_human_id
        return new_human_id + "_B"
    
    
df_bugs["human_id"] = df_bugs.apply(
    lambda row: get_human_id(
        short_repo_name=row["only_repo_name"], 
        msg=row["msg"]),
    axis=1
)

In [14]:
df_bugs.head()

,id,real,type,repo,commit_hash,component,symptom,bug_pattern,complexity,comment,localization,only_repo_name,msg,human_id
0,1,bug,Classical,Qiskit/qiskit-ignis,ec1b4ce759f1fb8ba0242dd6c4a309fa1b586666,Quantum State Evaluation,"Crash - OS/PL Error, Crash",Missing Error Handling,19,"From fixer: ""Currently, cvx_fit does not check...",https://github.com/Qiskit/qiskit-ignis/commit/...,qiskit-ignis,Fix to allow cvx_fit to retry and finally grac...,qiskit-ignis#3
1,10,bug,Quantum,Qiskit/qiskit-ignis,e696fee3e52550f50fb240a4c6356c61c9f01725,Quantum State Evaluation,Incorrect Output,Incorrect Numerical Computation,2,"Reporter: ""ignis.characterization.calibrations...",https://github.com/Qiskit/qiskit-ignis/commit/...,qiskit-ignis,fix FC rotation direction (#302),qiskit-ignis#302
2,1010,bug,Quantum,Qiskit/qiskit-aer,dc7a2413e6c373455c00cff51e37eaaa17a4a7ac,Optimizations,"Crash - Application Error, Crash",Barrier Related,6,"Fixer: ""Currently qubit truncation will fail f...",https://github.com/Qiskit/qiskit-aer/commit/dc...,qiskit-aer,Fix ReduceBarriers to be applied before trunca...,qiskit-aer#693
3,1012,bug,Classical,Qiskit/qiskit-aer,7de864cc989b354d0ad1febbd62b891270927b6b,Testing,Failing Test,"Flaky Test, Misconfiguration, Misconfiguration...",1,"Fix issue where the ""diagonal"" gate is checked...",https://github.com/Qiskit/qiskit-aer/commit/7d...,qiskit-aer,Fix tolerance when checking if diagonal is uni...,qiskit-aer#707
4,1019,bug,Quantum,Qiskit/qiskit-aer,f3b4303f478ff84bf8da15761827287d3a2f16e4,Simulator,Incorrect Output,"API Misuse - Internal, Typo",1,"Fixer: ""The new Diagonal terra gate was being ...",https://github.com/Qiskit/qiskit-aer/commit/f3...,qiskit-aer,Fix bugs in density matrix simulation method f...,qiskit-aer#767


In [15]:
from collections import Counter
Counter(df_bugs["real"])

Counter({'bug': 223, 'fp': 135})

In [16]:
all_repos = list(df_bugs["repo"].unique())
all_repos

['Qiskit/qiskit-ignis',
 'Qiskit/qiskit-aer',
 'PennyLaneAI/pennylane',
 'ProjectQ-Framework/ProjectQ',
 'microsoft/QuantumLibraries',
 'microsoft/qsharp-compiler',
 'QE-Lab/OpenQL',
 'quantumlib/Cirq',
 'aws/amazon-braket-sdk-python',
 'unitaryfund/mitiq',
 'aspuru-guzik-group/tequila',
 'qulacs/qulacs',
 'xanaduai/strawberryfields',
 'Qiskit/qiskit-terra',
 'eclipse/xacc',
 'rigetti/pyquil',
 'dwavesystems/dwave-system',
 'microsoft/qsharp-runtime']

In [17]:
# trial single repository
i = 0

i_repo_long_name = all_repos[i]
i_repo_name = i_repo_long_name.split("/")[-1]
i_repo_folder = os.path.join(REPO_FOLDER, i_repo_name)

df_repo_single = df_bugs[
    df_bugs["repo"] == i_repo_long_name
]

repo_commits = list(df_repo_single["commit_hash"])

i_repo_git = Repository(i_repo_folder, only_commits=repo_commits)

In [18]:
repo_commits

['ec1b4ce759f1fb8ba0242dd6c4a309fa1b586666',
 'e696fee3e52550f50fb240a4c6356c61c9f01725',
 '3c59f82c11e87c071bc7e84240b50e2aa995281f',
 'e8a4b14bc36dd6c1bbf11152c37e04fe597d4de7',
 '838bc0f750d63b4ea14789fd174f9a10f51f61c2',
 '640c2401b7d3821d77179a713b5ca14348a09b40',
 '13133b0e95dec69521a6b90e9dd83e17de2f6f51',
 '8f6aad2d6cf94ee33cbc8aaeb4afabad05a8cedd',
 '439b3ee739fde633337f89b617c18821b25da0ad',
 '0af27dd1fdd5c4e4d2ef80d21d0375d11a1625fd',
 'a136e72156a20c233446ebd6108038cba7ac9f1b',
 '824372e18a42a735bbef37cd8ee61bc5d5821f9f',
 '1f7b71999c28483064e91882ef1510ede23ba0e2',
 '864663d2f6212d893731ec2b5678592e4f9a4a02',
 '2f47127a99b709af2ea1f844aeee8f1d90e49194',
 '3694ca68abf54a03e73318e81d8932e6a7159f12',
 'cfcdb0db739f3497abae80898d1a9cdd2be46a37',
 '23d5ef8231d0bb95cb03ad9b1b62ee37757b24b3',
 'c4c66591c7bccc0d94bb830239611870f38a1887']

In [24]:
def save_modified_file(path, modified_files):
    """Save the file at the given path if not none.\
    
    Note that this will save a before and after subfolders."""
    
    folder_before = os.path.join(path, "before")
    folder_after = os.path.join(path, "after")
    
    Path(folder_before).mkdir(parents=True, exist_ok=True)
    Path(folder_after).mkdir(parents=True, exist_ok=True)
    
    for mf in modified_files:
        # create before version
        mf_before_path = os.path.join(folder_before, mf.filename)
        with open(mf_before_path, 'w') as out_file:
            content = mf.source_code_before
            content = content if content is not None else ""
            out_file.write(content)
        # after before version
        mf_after_path = os.path.join(folder_after, mf.filename)
        with open(mf_after_path, 'w') as out_file:
            content = mf.source_code
            content = content if content is not None else ""
            out_file.write(content)

            
def inspect_commit(pydriller_commit, commit_name=None, author_comment=None, bug_id=None):
    """Create a directory to store this commit and iterate the files."""
    c_hash = pydriller_commit.hash
    c_msg = pydriller_commit.msg
    c_project_name = pydriller_commit.project_name
    print(f"{c_project_name} => {c_hash}")
    # iterate over all modified files
    if commit_name is None: 
        commit_name = c_hash
    commit_path = os.path.join(OUT_ORIGINAL_COMMIT, c_project_name, commit_name)
    Path(commit_path).mkdir(parents=True, exist_ok=True)
    # save an identifier of the commit
    with open(os.path.join(commit_path, c_hash) + ".txt", 'w') as stub_file: 
        stub_file.write(c_msg)
        
    # save the comment resulted from the inspection
    if author_comment is not None and bug_id is not None:
        with open(os.path.join(commit_path, f"bug_{bug_id}_comment.md"), 'w') as stub_file: 
            stub_file.write(author_comment)
    
    save_modified_file(path=commit_path, modified_files=list(pydriller_commit.modified_files))
    return commit_path

#for c in i_repo_git.traverse_commits():
#    inspect_commit(c)
         

In [25]:
df_real_bugs = df_bugs[
    df_bugs["real"] == "bug"
]

# CREATE FOLDER STRUCTURE
Path(OUT_ORIGINAL_COMMIT).mkdir(parents=True, exist_ok=True)


df_real_bugs["local_file_path"] = df_real_bugs.apply(
    lambda row: inspect_commit(
        pydriller_commit=list(
            Repository(
                os.path.join(REPO_FOLDER, row["only_repo_name"]),
                single=row["commit_hash"]
            ).traverse_commits()
        )[0],
        commit_name=row["human_id"],
        author_comment=row["comment"],
        bug_id=row["id"]
    ),
    axis=1
)

qiskit-ignis => ec1b4ce759f1fb8ba0242dd6c4a309fa1b586666
qiskit-ignis => e696fee3e52550f50fb240a4c6356c61c9f01725
qiskit-aer => dc7a2413e6c373455c00cff51e37eaaa17a4a7ac
qiskit-aer => 7de864cc989b354d0ad1febbd62b891270927b6b
qiskit-aer => f3b4303f478ff84bf8da15761827287d3a2f16e4
qiskit-aer => 0eab74b57f2704ad0893132b6c5f214603550c39
pennylane => 01196202394aed89f43d3044d64eb9e41f7d583c
qiskit-aer => 48498dd0edc4e1ed1ee4bb287edd89541af104b2
qiskit-aer => 72674296de7de6cfdf812595be33a2f6f637a29f
pennylane => 474f337b176f706a180ac62d6c7840c01f604192
qiskit-aer => 7fe4f9eadc835705c25c13c753b8550b199efa44
qiskit-aer => 20c3b06c2464ce0342a920dfbe91c6a496707f7f
qiskit-aer => 759a01f43726d5146422141bfdcc83fc34c180d0
qiskit-aer => bd00cf0e317a262f9b5cc500a07505a8323bc792
qiskit-aer => 1cfa05e66c090ed6b57e7631b4935f2a7f5a6aad
qiskit-ignis => 3c59f82c11e87c071bc7e84240b50e2aa995281f
pennylane => c0d269e5f1eba2f9d033bd9b6a79c10a11f4228a
ProjectQ => 722104c40e9526e92a71a2161cb5f2d8757bcbfe
ProjectQ 

xacc => ad39126c9b6a835a88d6113fbf75d4ef5e590627
xacc => ad39126c9b6a835a88d6113fbf75d4ef5e590627
xacc => e5252e7a6560b83a12cd50bf59d839596c2939a7
xacc => 660c9125e0c4f1e6c833b5f2fba218be5350109c
xacc => 31b22ebe4608f1a1dbe60c99870af30cd6e15092
xacc => c60662652b46f30b791aeef0ac0728d27d125ce6
pyquil => b6ef90d22065c0b9c87788477a04f8b38f56c2eb
pyquil => 92cb256530f4793fee007b6c1e8c9db4a22a1a7f
pyquil => c147eb4f9417af72b5f396b2859506524ecf5bf4
pyquil => 604bf264ef522f721aa2ab7ee36dd007e095f732
pyquil => adf04c9b1aa074c67bb34ea06d7f18ee06bbdfd7
pyquil => b9f050c6e33a036c10d6f984c3f0b0f29836d2e0
pyquil => 58965c33b8314cda119ff36e46b705a03835d397
pyquil => 75a7c7428235e017ac6740e804cce64057bc3a95
pyquil => 279a29b432a6c12d013b26201fc53410fca4f48d
dwave-system => 4050d488f3d6ffd0a828d531127a0231527d2b6c
dwave-system => 38629d04e0e33a3a2c69ff5c85938aed49356ce1
dwave-system => e83f632479ec51e425f62305f58ecf284b557a7e
dwave-system => e8a1fefc86f560799ded9dfc19b54e70964db17b
dwave-system => d0f

<ipython-input-25-b11954331418>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_real_bugs["local_file_path"] = df_real_bugs.apply(
